In [10]:
import numpy as np
import tensorflow as tf

 **Variational Inference**
Variational inference is a method for approximating the posterior distribution of some variables $x$ given data $D$. For example $x$ could be natural images belonging to different classes and $D$ is then dataset that we use to build a model. Using Bayes rule we can write the posterior as $p(x|D) = p(D|x)p(x)/P(D)$. When the prior $p(x)$ is conjugate to the likelihood $p(D|x)$ we can calculate this exactly but in generally  we cannot always find this analytically because the normalizing constant $p(D)$ is intractable to compute. 

The key idea in variational inference is to pick another distribution $q(x)$ and optimise its parameters until it is similar to the posterior. We usually measure similarity using the KL divergence between q and p. 

- Explain lower bound
    - Two terms in LB: 1/KL of q(x) and p(x|D), 2/constant
    - We can't calculate the two terms individually - that is after all why we need an approximation 
    - But we can implicitly maximise the KL-divergence by maximising the LB
          
- Explain mean field method
    



We use neural networks as probabilistic encoders and decoders. We assume that both $p(\mathbf{x}|\mathbf{z})$ and $p(\mathbf{z}|\mathbf{x})$ are multivariate Gaussian distributions with a diagonal covariance. The encoder is a neural network which takes as input examples from our training set $\mathbf{x}$ and outputs the mean and log of the variance of the $p(\mathbf{z}|\mathbf{x})$. These parameters are used to convert samples drawn from the noise distribution $N(\mathbf{0},\mathbf{I})$, $\mathbf{\epsilon}$, into samples, $\mathbf{z}$, from $p(\mathbf{z}|\mathbf{x})$. These are then input to the decoder neural network which gives us the paramaters for $p(\mathbf{x}|\mathbf{z})$. This process is illustrated below.

<img src=VAE.svg style="width: 50%; height:50%"/>

As the diagram suggests that we can draw $L$ samples, where $L > 1$, from $p(\mathbf{z}|\mathbf{x})$ per datapoint $\mathbf{x}$ by drawing $L\times M$ samples $\mathbf{\epsilon}$ from $N(0,I)$ where $M$ is the number of training examples.

We will start off by implementing the simple, shallow network described in the paper, with just one hidden layer in each of the encoder and decoder. The function below takes a batch of input vectors and outputs two vectors each input vector, which we interpret as the parameters of a Gaussian distribution conditioned on the input. We can easily modify the architecture later for example to make it deeper or to use convolutional layers.

In [11]:
def vae_module(x, name, units):
    #Add initializers
    with tf.variable_scope(name):
        h = tf.layers.dense(inputs=x, units = units[0], activation=tf.tanh, name='h')
        print('h', h.shape)
        mu = tf.layers.dense(inputs=h, units = units[1], 
            activation = None, name='mu')
        log_sig2 = tf.layers.dense(inputs=h, units = units[1], activation = None, name='log_sig2')
    print('mu_f', mu.shape, 'log_sig2_f', log_sig2.shape)
    return mu, log_sig2

The network will be trained to minimise the following cost function:

$$\mathrm{L}(\theta, \phi; \mathbf{x}^{(i)}) = -D_{KL}(q_\phi(\mathbf{z}|\mathbf{x}^{(i)})||p_\theta(\mathbf{z}))
+ \frac{1}{L}\sum\limits_{l=1}^{L}\log p_\theta(\mathbf{x}^{(i)}|\mathbf{z}^{(i,l)})$$

As discussed earlier, it is possible to evaluate exactly the term $D_{KL}(q_\phi(\mathbf{z}|\mathbf{x}^{(i)})||p_\theta(\mathbf{z}))$ in some cases. In the present situation, where we are dealing with multivariate Gaussian distributions, this term turns to be:

$$\frac{1}{2}\sum\limits_{j=1}^{J}(1 + \log((\sigma_j^{(i)})^2) + (\mu_j^{(i)})^2 + (\sigma_j^{(i)})^2)$$

[What is $J$ above???]

The paper provides a derivation of this result but I have included a more detailed version here [???].

The following functions calculate the KL-divergence and the estimate of $E_{q_\phi(\mathbf{z}|\mathbf{x}^{(i)})}\left[\log p_\theta(\mathbf{x}^{(i)}|\mathbf{z})\right]$

In [12]:
def gaussian_kl_div(mu, log_sig2):
    log_kl = 1 + log_sig2 - mu**2 - tf.exp(log_sig2)
    print('log_kl', log_kl.shape)
    return 0.5*tf.reduce_sum(log_kl, axis=-1)#ascertain axis

def log_prob(x, log_sig_2, mu):
    lp = -(tf.reduce_sum(tf.ones_like(log_sig_2)*np.pi + log_sig_2, axis=-1) 
      + tf.reduce_sum((x-mu)**2, axis=-1)
      + tf.reduce_sum(1./tf.exp(log_sig_2), axis=-1))/2
    print('lp', lp.shape)
    lp = tf.reduce_mean(lp, axis=-1)
    return lp

Finally we can assemble the autoencoder

In [13]:
#Specify the dimensions 
xdim = 64
zdim = 10
hu = 200

x_shape = [xdim]*2
z_shape = [zdim]

M = 32
L = 7

In [16]:
# def vae(x_shape, z_shape, M=100, L=1):
tf.reset_default_graph()

#PLACEHOLDERS
x = tf.placeholder(dtype=tf.float32, name='x', shape=[M]+x_shape)
eps = tf.placeholder(dtype=tf.float32, name='x', shape=[M, L]+z_shape)

#ENCODER
x_flat = tf.reshape(x, [M,-1])
print('x', x.shape, 'eps', eps.shape)
params_z_x = vae_module(x_flat, 'encoder', [200, np.product(z_shape)])
##Make the parameters M x 1 x zdim to enable broadcasting 
##when multiplying with eps which has dimensions M x L x zdim
log_sig2_z_x, mu_z_x = [tf.expand_dims(param, axis=1) for param in params_z_x]
print('mu_z_x', mu_z_x.shape, 'log_sig2_z_x', log_sig2_z_x.shape)

#SAMPLE FROM P(Z|X) USING REPARAMETERISATION TRICK'
z = mu_z_x + log_sig2_z_x*eps
print('z', z.shape)

#DECODER
##Flatten z to use in decoder
z_flat = tf.reshape(z, [M*L,-1])
log_sig2_x_z, mu_x_z = vae_module(z_flat, 'decoder', [200, np.product(x_shape)])
# log_sig2_x_z, mu_x_z = [tf.reshape(param, [M, L, -1]) for param in params_x_z]
print('mu_x_z', mu_x_z.shape, 'log_sig2_x_z', log_sig2_x_z.shape)
log_sig2_z_x, mu_z_x = [tf.squeeze(param, axis=1) for param in [log_sig2_z_x, mu_z_x]]

#KL TERM
kl_term = gaussian_kl_div(mu_z_x, log_sig2_z_x)
print('kl_term', kl_term.shape)

#MC TERM
log_sig2_x_z, mu_x_z = [tf.reshape(param, [M, L, -1]) for param in [log_sig2_x_z, mu_x_z ]]
print('mu_x_z', mu_x_z.shape, 'log_sig2_x_z', log_sig2_x_z.shape)
##Expand x_flat as we expanded the parameters of p(z|x) above to allow broadcasting
x_expanded = tf.expand_dims(x_flat, axis=1)
print('x_expanded', x_expanded.shape)
mc_term = log_prob(x_expanded, mu_x_z, log_sig2_x_z)
print('mc_term', mc_term.shape)

#LOSS
lower_bound = tf.reduce_mean(mc_term - kl_term)
print('lower_bound', lower_bound.shape)

#OPTIMIZE
optim = tf.train.GradientDescentOptimizer(1e-2).minimize(lower_bound)

x (100, 64, 64) eps (100, 7, 10)
h (100, 200)
mu_f (100, 10) log_sig2_f (100, 10)
mu_z_x (100, 1, 10) log_sig2_z_x (100, 1, 10)
z (100, 7, 10)
h (700, 200)
mu_f (700, 4096) log_sig2_f (700, 4096)
mu_x_z (700, 4096) log_sig2_x_z (700, 4096)
log_kl (100, 10)
kl_term (100,)
mu_x_z (100, 7, 4096) log_sig2_x_z (100, 7, 4096)
x_expanded (100, 1, 4096)
lp (100, 7)
mc_term (100,)
lower_bound ()


In [17]:
num_epochs = 10
batch_size = 32

In [ ]:
with tf.session() as sess:
    for i in range(num_epochs):
        for j in range(math.ceil(num_examples/batch_size)):
            batch = data[j:j+batch_size]
            noise_samples = np.random.normal(loc=0,scale=1,size=[M, L]+z_shape)
            logvar_xz, mu_xz, lb = sess.run(feed_dict={x: batch, eps: noise_samples},
                                                             fetch=[log_sig2_x_z, mu_x_z, lower_bound])
        mu = mu_xz.reshape(-1)
        cov = np.diag(np.exp(logvar_xz).reshape(-1))
        images = np.random.multivariate_normal(mean=mu, cov=cov, size=5)
        images.reshape([xdim, -1]) #Assuming samples are of form [image, image, ..., image], image is 64 x 64
        plt.subplot(num_epochs, 1, i+1)
        plt.imshow(images)